In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)
import boto3
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
"""
    print datetime.today()
    d_app_list = []
    for device in [0,1,2]:
        db_df = query_df(daily_est_dsn, sql.format(device, date))
        d_app_list = d_app_list + list(db_df.app_id.unique())
    print datetime.today()
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().toPandas()
    print datetime.today()
    u_list = unified_df.app_id.unique()
    d_list = list(set(d_app_list))
    diff_list =  [x for x in  d_list if x not in u_list ]
    print datetime.today()
    if len(diff_list) ==  0:
        return "{} PASS".format(date)
    else:
        print diff_list
        return "{} FAIL: {} {} {}".format(date, len(u_list), len(d_list), diff_list[0:5])
        
        
def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))

s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/2019-2020/regression1.txt')
log = [] 

# 2010-07-04/   2020-02-21
date_list = ["2019-09-01"]# get_date_list("2019-02-19", "2019-05-31")
for date in date_list:
    temp_log = ""
    try:
        temp_log = compare(date)
        log.append(temp_log)
        print temp_log
    except Exception, e:
        temp_log = "{}: ERROR".format(date)
        print temp_log
        print e.message
    write_log(log, s3object)

